In [154]:
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import os

import numpy as np
import tensorflow as tf
import deepchem as dc
from deepchem.models.tensorgraph.models.graph_models import GraphConvTensorGraph
from deepchem.utils.save import load_from_disk
from deepchem.feat.graph_features import ConvMolFeaturizer

input_data='./gtop_smiles_processed_1.csv'

tasks = ['affinity_class']
featurizer=ConvMolFeaturizer()

loader = dc.data.CSVLoader(tasks=tasks, smiles_field="SMILES",featurizer=featurizer)
dataset=loader.featurize(input_data)


Loading raw samples now.
shard_size: 8192
About to start loading CSV from ./gtop_smiles_processed_1.csv
Loading shard 1 of size 8192.
Featurizing sample 0
TIMING: featurizing shard 0 took 0.216 s
TIMING: dataset construction took 0.251 s
Loading dataset from disk.


In [155]:
import deepchem as dc
import tempfile, shutil
from deepchem.splits.splitters import IndexSplitter,SingletaskStratifiedSplitter

splitter=SingletaskStratifiedSplitter(task_number=0)
train_data,valid_data,test_data=splitter.train_valid_test_split(dataset)



len(train_data),len(valid_data),len(test_data)

print(train_data)

TIMING: dataset construction took 0.033 s
Loading dataset from disk.
TIMING: dataset construction took 0.017 s
Loading dataset from disk.
TIMING: dataset construction took 0.017 s
Loading dataset from disk.


In [166]:
from deepchem.models.tensorgraph.models.graph_models import GraphConvTensorGraph

model = GraphConvTensorGraph(len(tasks), batch_size=20, dropout=0.5, mode='regression')

model.fit(train_data, nb_epoch=10)

/home/nagayasu/anaconda3/envs/deepchem/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:97: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Starting epoch 0
Starting epoch 1
Starting epoch 2
Starting epoch 3
Starting epoch 4
Starting epoch 5
Starting epoch 6
Starting epoch 7
Starting epoch 8
Starting epoch 9


9.499234747886657

In [167]:
from sklearn.metrics import mean_absolute_error
from deepchem.metrics import rms_score
import deepchem.hyper.grid_search

metric = dc.metrics.Metric(
    rms_score, task_averager=np.mean, mode="regression")

print("Evaluating model")
train_scores = model.evaluate(train_data, [metric])

valid_scores = model.evaluate(valid_data, [metric])

a = model.predict(valid_data)

Evaluating model
computed_metrics: [0.6019950209182436]
computed_metrics: [0.5973943111328058]


In [168]:
test_set=loader.featurize(input_data)

b = model.predict(test_set)

print(b)

np.savetxt("./test_set.csv",b,delimiter=",")

Loading raw samples now.
shard_size: 8192
About to start loading CSV from ./gtop_smiles_processed_1.csv
Loading shard 1 of size 8192.
Featurizing sample 0
TIMING: featurizing shard 0 took 0.214 s
TIMING: dataset construction took 0.246 s
Loading dataset from disk.
[[ 0.439663  ]
 [-0.213216  ]
 [ 0.00950846]
 [ 0.47989163]
 [ 0.4552202 ]
 [ 0.59154326]
 [ 0.25048068]
 [ 0.10327178]
 [ 0.7048497 ]
 [ 0.30651113]
 [-0.05924336]
 [-0.0902192 ]
 [ 0.4349831 ]
 [ 0.03276568]
 [ 0.38629183]
 [-0.062916  ]
 [ 0.51237494]
 [ 0.3881063 ]
 [ 0.32358196]
 [-0.10774159]
 [-0.13374181]
 [-0.28967202]
 [-0.09077056]
 [ 0.26112893]
 [ 0.3459794 ]
 [-0.09177215]
 [-0.08467265]
 [ 0.00703882]
 [ 0.02607217]
 [ 0.25048068]
 [-0.09177215]
 [-0.06212684]
 [ 0.45261624]
 [ 0.2166933 ]
 [ 0.84395534]
 [ 0.20000796]
 [-0.3577114 ]
 [ 0.5416581 ]
 [ 0.03388848]
 [ 0.12458321]
 [ 0.73287606]
 [ 0.54543567]
 [ 0.5511099 ]
 [-0.0972997 ]
 [-0.19380113]
 [-0.4332869 ]
 [ 0.5412897 ]
 [-0.04290161]
 [ 0.2197977 ]
